In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/Final Project/chest_xray.zip", 'r')

zip_ref.extractall("/tmp")
zip_ref.close()

In [17]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator

In [18]:
!ls "/tmp/chest_xray/"

test  train  val


In [19]:
base_dir = '/tmp/chest_xray/'

#the original dataset interchanged 'test' with 'validation,' with the latter being the 'holdout' dataset
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'val')

#training images
# train_n_dir = os.path.join(train_dir, 'normal')
# train_p_dir = os.path.join(train_dir, 'pneumonia')

#validation images
# val_n_dir = os.path.join(validation_dir, 'normal')
# val_p_dir = os.path.join(validation_dir, 'pneumonia')

In [20]:
train_datagen = ImageDataGenerator(rescale = 1.0/500. )
test_datagen = ImageDataGenerator(rescale = 1.0/500. )
val_datagen = ImageDataGenerator(rescale = 1.0/500. )


train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size=20,
                                                    color_mode='grayscale',
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
                                                    # shuffle=True,
                                                    # seed=42)


test_generator = test_datagen.flow_from_directory(test_dir, 
                                                 batch_size=20,
                                                 color_mode='grayscale',
                                                 class_mode='categorical',
                                                 target_size=(150, 150))
                                                #  shuffle=True,
                                                #  seed=42))


val_generator = test_datagen.flow_from_directory(validation_dir, 
                                                 batch_size=20,
                                                 color_mode='grayscale',
                                                 class_mode='categorical',
                                                 target_size=(150, 150))
                                                #  shuffle=True,
                                                #  seed=42))

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [22]:
# from keras.optimizers import RMSprop

# model.compile(optimizer=RMSprop(lr=0.001),
#               loss = 'binary_crossentropy',
#               metrics = ['acc'])

# from sklearn.utils import class_weight
# import numpy as np

# class_weights = class_weight.compute_class_weight(
#            'balanced',
#             np.unique(train_generator.classes), 
#             train_generator.classes)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



history = model.fit_generator(train_generator, validation_data=test_generator,
                    steps_per_epoch=50,
                    epochs=7,
                    validation_steps=50,
                    verbose=1)

Epoch 1/7
50/50 [==============================] - 53s 1s/step - loss: 0.5920 - accuracy: 0.7450 - val_loss: 0.7150 - val_accuracy: 0.6260
Epoch 2/7
50/50 [==============================] - 50s 1s/step - loss: 0.5245 - accuracy: 0.7680 - val_loss: 0.7803 - val_accuracy: 0.6291
Epoch 3/7
50/50 [==============================] - 50s 1s/step - loss: 0.3933 - accuracy: 0.7980 - val_loss: 0.4579 - val_accuracy: 0.7226
Epoch 4/7
50/50 [==============================] - 51s 1s/step - loss: 0.3205 - accuracy: 0.8740 - val_loss: 0.2268 - val_accuracy: 0.8636
Epoch 5/7
50/50 [==============================] - 51s 1s/step - loss: 0.1772 - accuracy: 0.9388 - val_loss: 0.4468 - val_accuracy: 0.7947
Epoch 6/7
50/50 [==============================] - 53s 1s/step - loss: 0.1493 - accuracy: 0.9430 - val_loss: 0.6730 - val_accuracy: 0.7614
Epoch 7/7
50/50 [==============================] - 51s 1s/step - loss: 0.1296 - accuracy: 0.9540 - val_loss: 0.4965 - val_accuracy: 0.7480


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
history_df = pd.DataFrame(history.history)

history_df

,val_loss,val_accuracy,loss,accuracy
0,0.680842,0.625000,0.594367,0.719000
1,0.504305,0.623967,0.550781,0.729000
2,0.749022,0.628049,0.447706,0.754016
3,0.633164,0.680785,0.372381,0.800000
4,0.732477,0.859756,0.292990,0.899000
5,0.652290,0.796488,0.164640,0.937000
6,0.185322,0.841463,0.190918,0.932000


In [ ]:
test_generator.reset()

pred = model.predict_generator(val_generator,verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (val_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=val_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results.to_string())

1/1 [==============================] - 0s 261ms/step
                                   Filename Predictions
0          NORMAL/NORMAL2-IM-1427-0001.jpeg   PNEUMONIA
1          NORMAL/NORMAL2-IM-1430-0001.jpeg      NORMAL
2          NORMAL/NORMAL2-IM-1431-0001.jpeg      NORMAL
3          NORMAL/NORMAL2-IM-1436-0001.jpeg   PNEUMONIA
4          NORMAL/NORMAL2-IM-1437-0001.jpeg      NORMAL
5          NORMAL/NORMAL2-IM-1438-0001.jpeg      NORMAL
6          NORMAL/NORMAL2-IM-1440-0001.jpeg      NORMAL
7          NORMAL/NORMAL2-IM-1442-0001.jpeg   PNEUMONIA
8   PNEUMONIA/person1946_bacteria_4874.jpeg      NORMAL
9   PNEUMONIA/person1946_bacteria_4875.jpeg   PNEUMONIA
10  PNEUMONIA/person1947_bacteria_4876.jpeg   PNEUMONIA
11  PNEUMONIA/person1949_bacteria_4880.jpeg      NORMAL
12  PNEUMONIA/person1950_bacteria_4881.jpeg      NORMAL
13  PNEUMONIA/person1951_bacteria_4882.jpeg   PNEUMONIA
14  PNEUMONIA/person1952_bacteria_4883.jpeg   PNEUMONIA
15  PNEUMONIA/person1954_bacteria_4886.jpeg   PNEUM

In [24]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import model_from_json

# save_path = '/content/drive/My Drive/Colab Notebooks/Final Project/models/'

# dot_img_file = '/content/drive/My Drive/Colab Notebooks/Final Project/images/simple_imbalanced.png'
# plot_model(model, to_file=dot_img_file, show_shapes=True)

results.to_csv('/content/drive/My Drive/Colab Notebooks/Final Project/models/simple_imbalanced/simple_imbal_eval.csv', index=False)
history_df.to_csv('/content/drive/My Drive/Colab Notebooks/Final Project/models/simple_imbalanced/simple_imbal_acc.csv', index=False)

model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/Final Project/models/simple_imbalanced/simple_imbal_model.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('/content/drive/My Drive/Colab Notebooks/Final Project/models/simple_imbalanced/simple_imbal_model.h5')